In [1]:
from pfs.utils.fiberids import FiberIds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
rlh = pd.read_csv('rlhoussaine.csv', index_col=0)

In [3]:
gfm = FiberIds()
gfmDf = pd.DataFrame(gfm.data)

scienceFiberId = np.arange(2394) + 1
scienceFiber = gfmDf.set_index('scienceFiberId').loc[scienceFiberId].reset_index()

In [4]:
class MTP(object):
    holePerRow = 8
    nHoles = 32
    spacing = 0.3
    
    @staticmethod
    def getHolePosition(xOrigin, yOrigin):
        x = np.array([i%MTP.holePerRow * MTP.spacing for i in range(32) ]) + xOrigin
        y = np.array([i//MTP.holePerRow * MTP.spacing for i in range(32)])*-1  + yOrigin
        return pd.DataFrame(dict(x=x,y=y))
    

class GangConnector(object):
    xDistMTP = 11.273
    yDistMTP = 6.85
    nMtpPerCol = 2
    
    @staticmethod
    def getHolePosition(df):
        dfs = [] 
        iMtp = 0
        for mtpName, perMtp in df.groupby('mtp', sort=False):
            mtpProperties = perMtp.copy()

            xOrigin = iMtp%GangConnector.nMtpPerCol * GangConnector.xDistMTP
            yOrigin = iMtp//GangConnector.nMtpPerCol * GangConnector.yDistMTP * -1
            mtpCoord = MTP.getHolePosition(xOrigin, yOrigin)
            
            mtpProperties['xGang'] = mtpCoord.loc[perMtp.mtpHoleId-1].x.to_numpy()
            mtpProperties['yGang'] = mtpCoord.loc[perMtp.mtpHoleId-1].y.to_numpy()
            dfs.append(mtpProperties)
            
            iMtp+=1
            
        return pd.concat(dfs)
            

In [5]:
res = []

for j, row in scienceFiber.iterrows():
    n, i, specId, mtpHole, mtpScience = row.mtp_A.split('-')
    res.append((row.fiberId, specId, f'{n}-{i}', n[0], int(mtpHole)))
    
res = pd.DataFrame(res, columns=['fiberId', 'specId', 'mtp', 'gang', 'mtpHoleId'])

In [6]:
dfs = []

for (specId, gang), df in res.groupby(['specId', 'gang']):
    holePosition = GangConnector.getHolePosition(df)
    holePosition['gang'] = gang
    dfs.append(holePosition)
    
dfs = pd.concat(dfs).sort_values('fiberId')

In [7]:
scienceFiber['xGang'] = dfs.xGang
scienceFiber['yGang'] = dfs.yGang
scienceFiber['gang'] = dfs.gang

In [8]:
sm3 = scienceFiber.query('spectrographId==3')

In [9]:
%matplotlib widget

In [10]:
plt.figure()

for gang, holePosition in sm3.groupby('gang'):

    plt.scatter(holePosition.xGang, holePosition.yGang, s=5., label=gang, alpha=0.4)
    
    
plt.gca().set_aspect('equal');
plt.legend(loc='best')

plt.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …